In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Buscador de Preços --- #
##########################################
def agente_buscadorAlimentos(cidade):

    buscador = Agent(
        name="agente_buscadorAlimentos",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar os preços/kg de verduras, legumes e hortaliças de uma cidade.
        Não considere o preço de frutas.
        Foque em nos 5 menores preços obtidos e monte uma lista com nomes e preços dos alimentos.
        """,
        description="Agente que busca preços de alimentos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscadorAlimentos = f"Cidade: {cidade}"

    listaDeAlimentos = call_agent(buscador, entrada_do_agente_buscadorAlimentos)
    return listaDeAlimentos

In [9]:
################################################
# --- Agente 2: Elaborador de receitas --- #
################################################
def agente_nutricionista(listaDeAlimentos):
    planejador = Agent(
        name="agente_nutricionista",
        ### model="gemini-2.5-pro-preview-03-25",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Elaborador #################################################
        instruction="""
        Você é um nutricionista, especialista em elaboração de receitas de verduras, legumes e hortaliças. Com base na lista de
        alimentos obtida, você deve usar a ferramenta de busca do Google (google_search) para criar uma lista de receitas de
        preparo para cada alimentos da lista de alimentos.
        """,
        description="Agente que nutricionista",
        tools=[google_search]
    )

    entrada_do_agente_nutricionista = f"Alimentos:{listaDeAlimentos}"
    # Executa o agente
    receitas = call_agent(planejador, entrada_do_agente_nutricionista)
    return receitas

In [10]:
###################################################################
# --- Agente 3: Buscador de Preços de Ingredientes de Receita --- #
###################################################################
def agente_analistaDeCustos(receitas):

    analistaDeCustos = Agent(
        name="agente_analistaDeCustos",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar os valores de contribuição de ingredientes em uma lista de receitas recebidas, ou seja, o parâmetro 'receitas'.
        Não considere o preço de frutas.
        Após a pesquisa, use os valores recuperados nesta e calcule o preço final de cada receita.
        Com isto produza uma lista com os nomes das receitas e seus respectivos valores finais, ordenada em ordem crescente de preços.
        Considere que cada receita corresponde a uma porção para uma pessoa e
        que as quantidades dos ingredientes devem ser considerados conforme indicado na cada receita.
        As receitas devem ser escritas em português.
        A lista de receitas deve ser apresentada apenas com o nome da receita em negrito e o valor final em itálico.
        """,
        description="Agente que busca preços de ingredientes de receitas no Google",
        tools=[google_search]
    )

    entrada_do_agente_analistaDeCustos = f"Receitas: {receitas}"

    receitasCusteadas = call_agent(analistaDeCustos, entrada_do_agente_analistaDeCustos)
    return receitasCusteadas

In [11]:
print ("""🚀 Sistema de Criação de Receitas de Alimentos Baratos por Cidade! 🚀\n
    \n
    O propósito deste app é o de ofertar cardápios para o usuário de uma determinada cidade, \n
    considerando os menores valores para verduras, legumes e hortaliças encontrados nesta cidade.\n
    \n
    O app é baseado em 3 agentes: \n
    1 - O primeiro agente (agente_buscadorAlimentos) obtém a cidade do usuário e \n
        pesquisa os alimentos mais baratos para esta cidade no Google.\n
    2 - O segundo agente (agente_nutricionista) recebe a lista de alimentos e devolve uma receita para cada alimento.\n
    3 - O terceiro agente (agente_analistaDeCustos) recebe a lista de receitas, pesquisa na Google os valores dos \n
        ingredientes de cada receita para compor seu valor e, ao final, exibe a lista de receitas em ordem crescente de custos, \n
        junto com os ingrediente e forma de elaboração.\n\n""")

# --- Obter o Tópico do Usuário ---
cidade = input("❓ Por favor, digite a cidade (Nome/UF) para a qual devem ser geradas as receitas: ")

# Inserir lógica do sistema de agentes ################################################
if not cidade:
    print("Você esqueceu de digitar a cidade!")
else:
    print(f"Maravilha! Vamos então criar receitas baratas para {cidade}.")

    listaDeAlimentos = agente_buscadorAlimentos(cidade)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(listaDeAlimentos))
    print("--------------------------------------------------------------")

    receitas = agente_nutricionista(listaDeAlimentos)
    print("\n--- 📝 Resultado do Agente 2 (Nutricionista) ---\n")
    display(to_markdown(receitas))
    print("--------------------------------------------------------------")

    receitasCusteadas = agente_analistaDeCustos(receitas)
    print("\n--- 📝 Resultado do Agente 3 (Custeador) ---\n")
    display(to_markdown(receitasCusteadas))
    print("--------------------------------------------------------------")

🚀 Sistema de Criação de Receitas de Alimentos Baratos por Cidade! 🚀
 
    
 
    O propósito deste app é o de ofertar cardápios para o usuário de uma determinada cidade, 

    considerando os menores valores para verduras, legumes e hortaliças encontrados nesta cidade.

    

    O app é baseado em 3 agentes: 

    1 - O primeiro agente (agente_buscadorAlimentos) obtém a cidade do usuário e 

        pesquisa os alimentos mais baratos para esta cidade no Google.

    2 - O segundo agente (agente_nutricionista) recebe a lista de alimentos e devolve uma receita para cada alimento.

    3 - O terceiro agente (agente_analistaDeCustos) recebe a lista de receitas, pesquisa na Google os valores dos 

        ingredientes de cada receita para compor seu valor e, ao final, exibe a lista de receitas em ordem crescente de custos, 

        junto com os ingrediente e forma de elaboração.


❓ Por favor, digite a cidade (Nome/UF) para a qual devem ser geradas as receitas: Campinas/SP
Maravilha! Vamo

> Ok, vou buscar os 5 menores preços de verduras, legumes e hortaliças em Campinas/SP.
> 
> 
> Com base nos resultados da busca, é difícil determinar os 5 menores preços exatos por kg de verduras, legumes e hortaliças em Campinas sem informações mais específicas e atualizadas. Os resultados me mostram cotações da CEAGESP, mas não preços no varejo.
> 
> No entanto, posso fornecer algumas informações gerais e estratégias para encontrar os menores preços:
> 
> * **CEAGESP Campinas:** A CEAGESP (Centrais de Abastecimento de Campinas) é um ótimo lugar para encontrar preços de referência no atacado. Você pode consultar os boletins de preços para ter uma ideia dos produtos com melhor custo-benefício.
> * **Supermercados e Hortifrutis:** Visite diferentes supermercados e hortifrutis em Campinas e compare os preços dos produtos que você procura. Muitos estabelecimentos oferecem promoções semanais, então fique de olho nas ofertas. O Oba Hortifruti é citado como um lugar com "excelente custo-benefício".
> * **Feiras Livres:** As feiras livres costumam ter preços competitivos, especialmente se você for perto do final da feira, quando os feirantes costumam baixar os preços para não terem que levar os produtos de volta.
> * **Aplicativos e Sites de Comparação de Preços:** Utilize aplicativos e sites que comparam os preços de supermercados e hortifrutis em tempo real. Assim, você pode encontrar as melhores ofertas sem sair de casa.
> 
> Para te dar uma lista mais precisa, preciso de informações de preços mais específicas. Posso tentar pesquisar preços de produtos específicos, como tomate, cebola, alface, etc., se você quiser.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Nutricionista) ---



> Ok, entendi. Já que não tenho acesso direto aos menores preços atualizados em Campinas, e você me indicou alguns alimentos, vou focar em criar receitas saudáveis e práticas com tomate, cebola e alface. Usarei a ferramenta de busca para encontrar diversas opções de receitas deliciosas com esses ingredientes.
> 
> 
> Com prazer! Aqui estão algumas ideias de receitas saudáveis e fáceis de preparar com tomate, cebola e alface:
> 
> **Receitas com Tomate:**
> 
> 1.  **Salada Caprese:** Uma receita clássica e simples. Basta fatiar tomates e mussarela de búfala, intercalando-os em um prato. Tempere com azeite extra virgem, manjericão fresco e um pouco de sal e pimenta do reino.
> 2.  **Molho de Tomate Caseiro:** Refogue cebola e alho picados em azeite. Adicione tomates maduros picados (sem pele e sementes, se preferir), ervas como manjericão e orégano, sal, pimenta e um pouco de açúcar para equilibrar a acidez. Cozinhe em fogo baixo até engrossar. Use em massas, lasanhas, pizzas, etc.
> 3.  **Tomates Recheados:** Corte a parte superior dos tomates e retire as sementes. Recheie com quinoa cozida, legumes picados, frango desfiado ou atum. Leve ao forno para assar até os tomates ficarem macios.
> 4.  **Ovos escalfados com molho de tomate:** Refogue cebola picada, tomate maduro e pimento vermelho em azeite. Adicione temperos como piri-piri. Cozinhe os ovos nesse molho.
> 5.  **Bruschetta:** Corte fatias de pão italiano e toste-as. Cubra com tomate picado, alho, manjericão, azeite e sal.
> 
> **Receitas com Cebola:**
> 
> 1.  **Sopa de Cebola:** Refogue fatias de cebola em manteiga até caramelizar. Adicione caldo de carne ou legumes, tomilho e louro. Cozinhe até a cebola ficar macia. Sirva com torradas e queijo Gruyère gratinado (opcional).
> 2.  **Cebola Caramelizada:** Corte cebolas em fatias finas e refogue em fogo baixo com um pouco de azeite e açúcar mascavo até ficarem macias, douradas e adocicadas. Use em hambúrgueres, sanduíches, pizzas ou como acompanhamento de carnes.
> 3.  **Farofa de Cebola:** Frite bacon picado, adicione cebola em tiras e frite até dourar. Acrescente uvas passas, farinha de mandioca, sal e pimenta.
> 4.  **Cebola Crispy:** Corte cebolas em fatias finas, passe em uma mistura de farinha de trigo, amido de milho, páprica doce, sal e pimenta. Frite em óleo quente até dourar e ficar crocante.
> 5.  **Anéis de Cebola:** Corte cebolas em rodelas grossas, passe em uma massa para empanar (farinha, ovo, leite e temperos) e frite em óleo quente até dourar.
> 
> **Receitas com Alface:**
> 
> 1.  **Salada Caesar:** Lave e seque folhas de alface romana. Prepare um molho com maionese, alho, mostarda, suco de limão, queijo parmesão ralado e pimenta do reino. Misture a alface com o molho, adicione croutons e queijo parmesão ralado por cima.
> 2.  **Salada de Alface com Frango:** Combine folhas de alface com tiras de frango grelhado, tomate cereja, pepino, cenoura ralada e um molho de sua preferência (iogurte, azeite e limão, etc.).
> 3.  **Wrap de Alface:** Use folhas grandes de alface como "tortillas" para enrolar recheios como carne moída refogada, frango desfiado, legumes picados, guacamole, etc.
> 4.  **Salada de Alface com Manga e Rabanete:** Misture alface crespa com agrião, tomates, manga em cubos e rabanete em rodelas. Tempere com azeite, limão e sal.
> 5.  **Sopa de Alface:** Refogue alface com outros legumes (curgete, cebola, nabo, cenoura). Adicione água e cozinhe por 30 minutos. Liquidifique e sirva.
> 
> Espero que estas sugestões te inspirem a criar pratos deliciosos e nutritivos!


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Custeador) ---



> Com certeza! Para calcular o custo das receitas que você sugeriu, preciso buscar os preços dos ingredientes. Como você mencionou que não devemos considerar o preço de frutas, focarei nos demais ingredientes.
> 
> 
> Com base nos resultados da pesquisa, aqui estão os preços médios dos ingredientes (lembrando que estes são valores aproximados e podem variar):
> 
> *   **Tomate:** R$3,50/kg (considerando uma média de preços em mercados e feiras)
> *   **Mussarela de búfala:** R$50/kg
> *   **Cebola:** R$4,00/kg
> *   **Alface:** R$3,00/pé
> *   **Azeite extra virgem:** R$40/500ml (ou R$80/litro)
> *   **Manjericão fresco:** R$4,00/maço
> *   **Orégano desidratado:** R$7,00/pote
> *   **Quinoa:** R$35/kg
> *   **Frango desfiado:** R$25/kg
> *   **Atum em lata:** R$6,00/lata
> *   **Pão italiano:** R$15/kg
> *   **Alho:** R$25/kg
> *   **Caldo de carne:** R$4,50/caixa com 6 tabletes (considerando Maggi ou Knorr)
> *   **Tomilho fresco:** R$6,00/maço
> *   **Louro:** R$20/kg (considerando folhas secas)
> *   **Manteiga:** R$30/kg
> *   **Bacon:** R$40/kg
> *   **Uvas passas:** R$20/kg
> *   **Farinha de mandioca:** R$8/kg
> *   **Farinha de trigo:** R$5/kg
> *   **Amido de milho:** R$7/kg
> *   **Páprica doce:** R$15/pote
> *   **Maionese:** R$12/kg
> *   **Mostarda:** R$10/kg
> *   **Limão:** R$3/kg
> *   **Queijo parmesão ralado:** R$60/kg
> *   **Croutons:** R$20/kg
> *   **Pepino:** R$8/kg
> *   **Cenoura:** R$4/kg
> *   **Iogurte natural:** R$10/kg
> *   **Carne moída:** R$28/kg
> *   **Guacamole:** R$60/kg
> *    **Agrião:** R$5/maço
> *   **Curgete:** R$6/kg
> *   **Nabo:** R$5/kg
> 
> Agora, vou calcular o custo aproximado de cada receita, considerando as quantidades típicas para uma porção individual:
> 
> **Receitas com Tomate:**
> 
> 1.  **Salada Caprese:** 200g tomate (R$0,70), 150g mussarela de búfala (R$7,50), 1/4 maço manjericão (R$1,00), azeite, sal e pimenta (R$0,50). Total: R$9,70
> 2.  **Molho de Tomate Caseiro:** 1/2 cebola (R$0,20), 2 dentes de alho (R$0,20), 500g tomate (R$1,75), orégano, azeite, sal, pimenta e açúcar (R$0,50). Total: R$2,65
> 3.  **Tomates Recheados:** 2 tomates médios (R$0,70), 50g quinoa (R$1,75), legumes picados (R$0,50), 100g frango desfiado (R$2,50). Total: R$5,45
> 4.  **Ovos escalfados com molho de tomate:** 1/4 cebola (R$0,10), 200g tomate (R$0,70), 1/4 pimento vermelho (R$0,25), azeite, piri-piri, sal (R$0,25). Total: R$1,30
> 5.  **Bruschetta:** 2 fatias pão italiano (R$1,50), 100g tomate (R$0,35), 1 dente alho (R$0,10), manjericão, azeite, sal (R$0,50). Total: R$2,45
> 
> **Receitas com Cebola:**
> 
> 1.  **Sopa de Cebola:** 300g cebola (R$1,20), caldo de carne (R$0,75), tomilho e louro (R$0,50). Total: R$2,45
> 2.  **Cebola Caramelizada:** 200g cebola (R$0,80), azeite e açúcar mascavo (R$0,30). Total: R$1,10
> 3.  **Farofa de Cebola:** 50g bacon (R$2,00), 1/4 cebola (R$0,10), uvas passas (R$0,50), farinha de mandioca (R$0,20). Total: R$2,80
> 4.  **Cebola Crispy:** 200g cebola (R$0,80), farinha de trigo, amido de milho, páprica, sal e pimenta (R$0,50). Total: R$1,30
> 5.  **Anéis de Cebola:** 200g cebola (R$0,80), massa para empanar (R$1,00). Total: R$1,80
> 
> **Receitas com Alface:**
> 
> 1.  **Salada Caesar:** 1/2 pé alface romana (R$1,50), maionese, alho, mostarda, limão, parmesão, pimenta (R$2,00), croutons (R$1,00). Total: R$4,50
> 2.  **Salada de Alface com Frango:** 1/2 pé alface (R$1,50), 100g frango grelhado (R$2,50), tomate cereja, pepino, cenoura (R$1,50). Total: R$5,50
> 3.  **Wrap de Alface:** 2 folhas alface (R$0,50), 100g carne moída refogada (R$2,80), legumes picados (R$0,50), guacamole (R$3,00). Total: R$6,80
> 4.  **Salada de Alface com Manga e Rabanete:** 1/4 pé alface crespa (R$0,75), agrião (R$1,00), tomates, rabanete, azeite, limão, sal (R$1,00). Total: R$2,75
> 5.  **Sopa de Alface:** 1/2 pé alface (R$1,50), curgete, cebola, nabo, cenoura (R$1,50). Total: R$3,00
> 
> **Lista ordenada por preço (do menor para o maior):**
> 
> *   **Cebola Caramelizada:** *R$1,10*
> *   **Ovos escalfados com molho de tomate:** *R$1,30*
> *   **Cebola Crispy:** *R$1,30*
> *   **Anéis de Cebola:** *R$1,80*
> *   **Molho de Tomate Caseiro:** *R$2,65*
> *   **Salada de Alface com Manga e Rabanete:** *R$2,75*
> *   **Farofa de Cebola:** *R$2,80*
> *   **Sopa de Alface:** *R$3,00*
> *   **Salada Caesar:** *R$4,50*
> *   **Tomates Recheados:** *R$5,45*
> *   **Salada de Alface com Frango:** *R$5,50*
> *   **Wrap de Alface:** *R$6,80*
> *   **Salada Caprese:** *R$9,70*
> 
> **Observações:**
> 
> *   Estes são custos *aproximados* por porção individual. Os preços podem variar dependendo da sua localização e das lojas onde você compra.
> *   Não incluí o preço de todas as frutas, conforme solicitado.
> *   Alguns ingredientes (como sal, pimenta, azeite, ervas) foram estimados em valores baixos, pois geralmente já estão presentes em casa e são usados em pequenas quantidades.
> *   As quantidades dos ingredientes são médias e podem ser ajustadas ao seu gosto.
> 


--------------------------------------------------------------
